In [1]:
import pandas as pd
import numpy as np

In [2]:
purchase_info = pd.read_pickle("./how_to_make/purchase.pkl")
person_info = pd.read_csv("./dataset/LPOINT_BIG_COMP_01_DEMO.csv", low_memory=False)

In [121]:
# 추천시스템 적용할 cust입력
person_id = "M355489415"
# person_info 데이터에서 person_id를 적용
cust_info = person_info[person_info['cust'] == person_id]
# 성별과 나이를 문자열로 추출
gender = cust_info['ma_fem_dv'].values[0]
age = cust_info['ages'].values[0]
# person_id가 일치하는 사람이 많이 구매한 물품 3개
perfer_purchase = purchase_info[purchase_info['cust'] == person_id].sort_values(['buy_ct'], ascending=False).drop_duplicates(['pd_nm'])['pd_nm'][:3].values

In [114]:
# 비교할 같은 성별과 나이의 모든 사람의 cust를 추출
other_person = person_info[(person_info['ma_fem_dv'] == gender) & (person_info['ages'] == age)]['cust'].values
# 모든 구매내역에서 other_person에 저장된 값만 추출
other_person_goods = purchase_info[purchase_info['cust'].isin(other_person)][['pd_c','pd_nm','clac_hlv_nm', 'clac_mcls_nm']].drop_duplicates(['pd_nm']).reset_index()

In [115]:
# other_person_goods에서 중복값을 뺀 데이터로 코사인 유사도 측정
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))
c_vector_genres = count_vector.fit_transform(other_person_goods['clac_mcls_nm'])
from sklearn.metrics.pairwise import cosine_similarity
genres_cosine = cosine_similarity(c_vector_genres,c_vector_genres).argsort()[:,::-1]

In [127]:
# 추천리스트를 뽑아주는 함수
def get_recommend_movie_list(df, goods_name, top=4):
    target_movie_index = df[df['pd_nm'] == goods_name].index.values
    sim_index = genres_cosine[target_movie_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != target_movie_index]
    result = df.iloc[sim_index]
    return result['pd_nm']

In [129]:
for i in perfer_purchase:
    print(f"구매 상품: {i}")
    print("추천상품: ")
    print(get_recommend_movie_list(other_person_goods, goods_name=i))

구매 상품: 모나카
추천상품: 
227     컵아이스크림
193         얼음
419    튜브아이스크림
Name: pd_nm, dtype: object
구매 상품: 바아이스크림
추천상품: 
196        모나카
227     컵아이스크림
193         얼음
419    튜브아이스크림
Name: pd_nm, dtype: object
구매 상품: 스포츠음료
추천상품: 
1366     식이섬유음료
260      숙취해소음료
1078       한방음료
752     기타기능성음료
Name: pd_nm, dtype: object
